# Week 13: Example Sentences Classification
In the last two assignment, we use deep learning method to do classification task. We can get good result by deep learning, but we can hardly explain the classification result(i.e., we don't know why the phrase is classified as "good" phrase). Hence, in this assignment, we want you do classification again, but do it by traditional machine learning method. In this way, you can know *why* more. We want you implement this on example sentences(e.g., "This song has a brilliant piano intro." for word "brilliant").

[Here](https://drive.google.com/drive/folders/1ij20ecLlI1Zh5CdMAa91SXQzmpKfdBdJ?usp=sharing) are two files needed for this task: **train.jsonl** & **test.jsonl**. In these files, each line records one example sentence with its label. There're two types of labels: ***positive*** means it's *good* example sentence;  ***negative*** means it's *bad* example sentence.

In [ ]:
"""mount Google Drive to Colab directory
"""
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/week13/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/week13


## Read Data
We use dataframe to keep data in this assignment.

In [ ]:
import json
import pandas as pd

In [ ]:
def read_data_to_df(path):
    labels = []
    sentences = []
    with open(path, 'r') as f:
        for line in f.readlines():
            line = json.loads(line)
            sentences.append(line['text'])
            labels.append(line['label'])
    return pd.DataFrame({'sent': sentences,'label': labels})  

In [ ]:
train = read_data_to_df('data/train.jsonl')
print(train.head())
test = read_data_to_df('data/test.jsonl')
print(test.head())

                                                sent     label
0         My children threw a birthday party for me.  positive
1  Marketing on social networking sites is a fast...  positive
2                You pillock, look what you've done!  positive
3      He scored three touchdowns early in the game.  positive
4          His abrupt manner makes me uncomfortable.  positive
                                                sent     label
0  I've just spotted Mark - he's over there, near...  positive
1  After repricing, the bonds yield a much higher...  positive
2             I admire her staunchness and fidelity.  positive
3  The party's leader is in favour of the treaty ...  positive
4  About 20 companies are working on treatments f...  positive


## Extract Features
Traditional machine learning need data scientists to observe data and find out some useful information. 

Here is an example:

In [ ]:
import re
# for positive
spec = 0
count = 0
for s in train[train['label'] == "positive"]["sent"]:
    if len(re.sub('[\w\s]', '', s)) > 3:
        spec += 1
    count += 1
print(spec / count)

0.058014915590512126


In [ ]:
spec = 0
count = 0
for s in train[train['label'] == "negative"]["sent"]:
    if len(re.sub('[\w\s]', '', s)) > 3:
        spec += 1
    count += 1
print(spec / count)

0.3834226149596325


After some experiment, we found 38% of bad example sentences have more than 3 punctuations. In contrast, only 5.8% of good example sentences have more than 3 punctuations. Hence, it seems that it is a nice feature to distinguish good and bad example sentences, so we add this feature.

In [ ]:
train["more than three punc"] = [1 if len(re.sub('[\w\s]', '', s)) > 3 else 0 for s in train["sent"]]
test["more than three punc"] = [1 if len(re.sub('[\w\s]', '', s)) > 3 else 0 for s in test["sent"]]

In [43]:
train.head()

,sent,label,normally capitalized,no special symbols,more than three words,length of sentence
0,My children threw a birthday party for me.,positive,1,1,1,1
1,Marketing on social networking sites is a fast...,positive,1,1,1,1
2,"You pillock, look what you've done!",positive,1,1,1,1
3,He scored three touchdowns early in the game.,positive,1,1,1,1
4,His abrupt manner makes me uncomfortable.,positive,1,1,1,1


<font color="red">**[ TODO ]**</font> Please observe the data and extract at least three features and add them to the dataframe.

In [42]:
# first character is capitalized, others aren't (except for 'I')
train["normally capitalized"] = [1 if s[0].isupper() and all(c.islower() or c == 'I' for c in s[1:] if c.isalpha()) else 0 for s in train["sent"]]
test["normally capitalized"] = [1 if s[0].isupper() and all(c.islower() or c == 'I' for c in s[1:] if c.isalpha()) else 0 for s in test["sent"]]

# should not use any weird symbol or different country's language
train["no special symbols"] = [1 if all(c.isalnum() or c in "€£–',. -%$/?!’" for c in s) else 0 for s in train["sent"]]
test["no special symbols"] = [1 if all(c.isalnum() or c in "€£–',. -%$/?!’" for c in s) else 0 for s in test["sent"]]

# length of sentence is important
train["length of sentence"] = [1 if len(s) >= 20 else 0 for s in train["sent"]]
test["length of sentence"] = [1 if len(s) >= 20 else 0 for s in test["sent"]]

# words in the sentence should be large enough
train["more than three words"] = [1 if len(s.split()) > 3 else 0 for s in train["sent"]]
test["more than three words"] = [1 if len(s.split()) > 3 else 0 for s in test["sent"]]

## Train
Now, it's time to evaluate whether the features just selected is useful to classify. We use [Bernoulli Naive Bayes model](https://scikit-learn.org/stable/modules/naive_bayes.html#bernoulli-naive-bayes) to train training data. 

In [ ]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()

<font color="red">**[ TODO ]**</font> Please adjust the `selected_features` list and train the model.

***Don't*** use other model in this assignment.

In [44]:
selected_features = ["normally capitalized", "no special symbols", "length of sentence", "more than three words"]

In [45]:
y_pred = bnb.fit(train[selected_features], train['label']).predict(test[selected_features])

## Test
Test the model and get the accuracy of the prediction on testing data. 

There're four baselines for this task:
```
1. simple baseline: 0.65
2. medium baseline: 0.72
3. strong baseline: 0.8
4. boss baseline: 0.85
```
The more the baseline you pass, the more higher the grade you can get.

*hint: If the result isn't ideal, you can print the wrong prediction data and re-observe but only focus on wrong data to extract other features.

In [46]:
print((test['label'] == y_pred).sum()/len(test))  # 0.8006553079947575

0.8006553079947575


## TA's Notes

If you complete the Assignment, please use [this link](https://docs.google.com/spreadsheets/d/1QGeYl5dsD9sFO9SYg4DIKk-xr-yGjRDOOLKZqCLDv2E/edit#gid=1031097651) to reserve demo time.  
The score is only given after TAs review your implementation, so <u>**make sure you make a appointment with a TA before you miss the deadline**</u> .  <br>After demo, please upload your assignment to eeclass. You just need to hand in this ipynb file and rename it as XXXXXXXXX(Your student ID).ipynb.
<br>Note that **late submission will not be allowed**.